In [ ]:
import reobase_analysis.exper_utils as xu
import reobase_analysis.build_table as bt
import matplotlib.pylab as plt
import reobase_analysis.sin_utils as su
import reobase_analysis.table_plot_helper as tpl
import isee_engine.bionet.config as config
import numpy as np
import pandas as pd

# Build tables

For this part, you need to have  
1) a json file in the following folder: "/allen/aibs/mat/sooyl/Stimulus_Item_Values"  
2) you need to have the experimental .nwb files in "/allen/aibs/mat/sooyl/50Khz_nwb_files"

In [ ]:
experiment_id = ['2018_10_24_133422']
sampling_freq = 50
# Any frequency lower than lowcut_freq and higher than highcut_freq is discarded from Vext
lowcut_freq = 0.5 
highcut_freq = 200 
saved_data = False

for ID in experiment_id:
     bt.build_expr_table(ID, sampling_freq, lowcut_freq, highcut_freq)

# Read the table

#### You can read the table by providing the path or by using the experiment_id. This is by providing the path:

In [ ]:
xu.read_table_h5("/allen/aibs/mat/sooyl/result_tables/table_2018_10_24_133422_50.h5")

#### This is by providing the experiment_id:

In [ ]:
saved_data = False
experiment_id = '2018_10_24_133422'
table = xu.read_table_from_exp_id(experiment_id, sampling_freq, saved_data)
table


# Looking at the control experiments

All the analysis for control experiment is done the same way as for the other experiment. However for the spike phase analysis, controls dont have any sin wave. Therefore we computed the spike phase based on the other sweeps with different frequencies after them. In the following table, you can see the data related to the control experiments. 

In [ ]:
print table[table['ex_amp(nA)']==0]['in_dur(ms)'].unique()
print table[table['ex_amp(nA)']==0]['ex_dur(ms)'].unique()

In [ ]:
table[table['ex_amp(nA)']==0]

In [ ]:
table[table['sweep_number']==67]

For any specific sweep, you can plot the Vi and Vext trace as the following:

In [ ]:
sweep = 57
ex_el_id = 7
in_el_id = 5 

# Reading the Vext 
v = xu.read_trace_from_nwb(experiment_id, sampling_freq,  sweep, ex_el_id, saved_data)
tpl.plot_nwb_trace(v, sampling_freq, title="Raw Ve trace")


# Reading the Vi
v = xu.read_trace_from_nwb(experiment_id, sampling_freq,  sweep, in_el_id, saved_data)
tpl.plot_nwb_trace(v, sampling_freq, title="Vi trace")
plt.show()

print "in the table there are", table[(table['sweep_number'] == sweep) & (table['ex_el_id'] == ex_el_id)]["num_spikes"][0] ,"spikes for this sweep" 

# Let's do some checks on the table
## 1- Check the presence of spike in the table and compare it with the raw data
You can use the table to get the extracellular and intracellular electrode_id and then use that to get the raw data from nwb file and check for example to see if in the table there is spike for a specific sweep, do the raw data also shows correctly if there is spike. For example in the below, we can look at sweep=181, which has extracellular_electrode_id = 7 and intracellular_electrode_id =5. In the table, we see that sweep, has spikes. Now we want to look at the raw trace and see if there is spike.

In [ ]:
s = table['sweep_number'].unique()

In [ ]:
# ex_el_id = [table['ex_el_id'].unique()[0]]
# in_el_id = [table['in_el_id'].unique()[0]]  


# for sweep in s:
#     for ex_el in ex_el_id:
#         v = xu.read_trace_from_nwb(experiment_id, sampling_freq,  int(sweep), ex_el, saved_data)
#         v = xu.read_trace_from_nwb(experiment_id, sampling_freq,  int(sweep), in_el_id[0], saved_data)
    
# print "All the simulus descriptions are correct"

In [ ]:
table[table['num_spikes']>0]

In [ ]:
sweep = 76
ex_el_id = 7
in_el_id = 5 

# Reading the Vext 
v = xu.read_trace_from_nwb(experiment_id, sampling_freq,  sweep, ex_el_id, saved_data)
tpl.plot_nwb_trace(v, sampling_freq, title="Raw Ve trace")


# Reading the Vi
v = xu.read_trace_from_nwb(experiment_id, sampling_freq,  sweep, in_el_id, saved_data)
tpl.plot_nwb_trace(v, sampling_freq, title="Vi trace")
plt.show()

print "in the table there are", table[(table['sweep_number'] == sweep) & (table['ex_el_id'] == ex_el_id)]["num_spikes"][0] ,"spikes for this sweep" 

#### You can do the above check for different electrodes and see if it shows spike data in the table, the raw data also contains spikes.

## 2- Check the Vext and Vi phase and amplitude
It is important to note that if there is no spike and if there is a sin extracellualr current present, then both Vi and Vext should have amplitude and phase values inside the table. I call an experiment a control, when the extracellualr stimulation is =0. In this case all the values printed below should be NAN. Which means that there is no sin wave anywhere. So the outcome the following test MUST be TRUE. 

In [ ]:
table[table['ex_amp(nA)']==0][['vext_amp(mV)', 'vext_phase', 'vi_amp(mV)','vi_phase', 'vm_amp(mV)', 'vm_phase']].isnull().values.all()

For all other sweeps except the control, we MUST have values for Vext_amp and Vext_phase, so we MUST get FALSE for the following line:

In [ ]:
table[table['ex_amp(nA)']!=0][['vext_amp(mV)', 'vext_phase']].isnull().values.all()

However for vi, the vi_amp and vi_phase are computed when the cell is not spiking. So if there is spike, then these values are NAN. so the output of the follwoing test MUST be TRUE.

In [ ]:
table[table['num_spikes']>0][['vi_amp(mV)', 'vi_phase']].isnull().values.all()

Vm is computed only when there is not spike and only for the closest electrode because Vext of the closest electrode must be used in this function: Vm= Vi- Vext. Therefore the outcome of the following tests must be TRUE.

In [ ]:
print table[table['num_spikes']>0][['vm_amp(mV)', 'vm_phase']].isnull().values.all()
print ~table[(table['ex_el_distance(mu)']==50) & (table['num_spikes']==0)][['vm_amp(mV)', 'vm_phase']].isnull().values.all()

In [ ]:
table[(table['num_spikes']>0)]

In [ ]:
table[table['sweep_number']==71][['vext_amp(mV)', 'vext_phase','vi_amp(mV)', 'vi_phase']]

Here I am checking the phase and amplitude of Vext for one sweep. For this specific sweep, Vi amplitude and phase are NAN becasue the cell is spiking but the Vext amplitude and phase are not NAN and in the below, I show how the values are computed.

In [ ]:
sweep = 71
electrode_id = table['ex_el_id'].unique().tolist()
freq = 31 
ex_dur = 13000
ex_delay = 1000

for el in electrode_id:
    #Find the signal from NWB file
    v = xu.read_trace_from_nwb(experiment_id, sampling_freq,  sweep, el, saved_data)
    #Filter the signal
    fil_v = su.bandpass_filter(v, 0.5, 110, 1./(sampling_freq * 1000.))
    #Plot the signal andfiltered signal
    tpl.plot_nwb_trace(v, 50, title="Raw trace")
    tpl.plot_nwb_trace(fil_v, 50, title="Filtered trace")

    # Compute the mean trace and plot the mean trace and its amplitude
    sin = su.get_fitted_sin(fil_v,ex_delay, ex_dur, freq, 0.02)
    mean_fil_v_trace = su.get_mean_trace(var_trace = fil_v, ex_delay= ex_delay, ex_dur = ex_dur, freq = freq, dt = 0.02) 
    amplitude_line = table[(table['sweep_number']==sweep) & (table['ex_el_id']==el)]['vext_amp(mV)'][0]
    ax1 = tpl.plot_nwb_trace(np.repeat(amplitude_line, len(mean_fil_v_trace)), 50)
    tpl.plot_nwb_trace(mean_fil_v_trace, 50, title="Mean trace", ax= ax1)
    tpl.plot_nwb_trace(sin, 50, ax=ax1)

plt.show()


We can do the same test for another sweep in which Vi is not spiking and therefore we can compute the phase and amplitude if Vi for example the following sweep:

In [ ]:
# table[(table['num_spikes']==0)]['in_amp(pA)'].unique()
table[(table['num_spikes']==0) & (table['ex_amp(nA)']!=0)]

In [ ]:
sweep = 79
ex_electrode_id = 7
print table[(table['sweep_number']==sweep) & (table['ex_el_id']==ex_electrode_id)]['vm_amp(mV)'][0]
print table[(table['sweep_number']==sweep) & (table['ex_el_id']==ex_electrode_id)]['vi_amp(mV)'][0]


In [ ]:
sweep = 79
in_electrode_id = 5 
ex_electrode_id = 7
freq = 8 
ex_dur = 9000
ex_delay = 1000


#Find the signal from NWB file
vi = xu.read_trace_from_nwb(experiment_id, sampling_freq,  sweep, in_electrode_id, saved_data)

ve = xu.read_trace_from_nwb(experiment_id, sampling_freq,  sweep, ex_electrode_id, saved_data)
fil_ve = su.bandpass_filter(ve, 0.5, 110, 1./(sampling_freq * 1000.))

vm =  vi - (fil_ve- np.mean(fil_ve))

amplitude_line = table[(table['sweep_number']==sweep) & (table['ex_el_id']==ex_electrode_id)]['avg_vm(mV)'][0]
ax = tpl.plot_nwb_trace(vi, 50, title="Raw trace", label="Vi")
tpl.plot_nwb_trace(vm, 50,  ax=ax, label="Vm")
tpl.plot_nwb_trace(fil_ve+amplitude_line, 50,  ax=ax, label="Ve")
tpl.plot_nwb_trace(np.repeat(amplitude_line, len(vm)), 50,  ax=ax)
# plt.ylim(amplitude_line-3, amplitude_line+3)
plt.xlim(3,8)
plt.show()


In [ ]:
sweep = 77
in_electrode_id = 5 
freq = 8
ex_dur = 9000
ex_delay = 1000


#Find the signal from NWB file
v = xu.read_trace_from_nwb(experiment_id, sampling_freq,  sweep, in_electrode_id, saved_data)
tpl.plot_nwb_trace(v, 50, title="Raw trace")
plt.show()

# Compute the mean trace and plot the mean trace and its amplitude
sin = su.get_fitted_sin(v,ex_delay, ex_dur, freq, 0.02)
mean_fil_v_trace = su.get_mean_trace(var_trace = v, ex_delay= ex_delay, ex_dur = ex_dur, freq = freq, dt = 0.02) 
amplitude_line = table[(table['sweep_number']==sweep) & (table['in_el_id']==in_electrode_id)]['vi_amp(mV)'][0]
ax1 = tpl.plot_nwb_trace(np.repeat(amplitude_line, len(mean_fil_v_trace)), 50)
tpl.plot_nwb_trace(mean_fil_v_trace, 50, title="Mean trace", ax= ax1)
tpl.plot_nwb_trace(sin, 50, ax=ax1)
plt.show()

In [ ]:
table[table['sweep_number']==77][['vext_amp(mV)', 'vext_phase','vi_amp(mV)', 'vi_phase']]

## 3- Check the Spike phase


In [ ]:
table[table['num_spikes']>0]

In [ ]:
table[table['sweep_number']==65]

First, lets look at the filtered signal and the hilbert phase for all the extracellular electrodes for one sweep.

In [ ]:
sweep = 65
electrode_id = table['ex_el_id'].unique().tolist()
# freq = 9
ex_dur = 13000
ex_delay = 1000

ax0 = plt.figure(figsize=(20,5))
ax0 = plt.subplot(111)

ax1 = plt.figure(figsize=(20,5))
ax1 = plt.subplot(111)

ax2 = plt.figure(figsize=(20,5))
ax2 = plt.subplot(111)


for el in electrode_id:
    #Find the signal from NWB file
    v = xu.read_trace_from_nwb(experiment_id, sampling_freq,  sweep, el, saved_data)
    tpl.plot_nwb_trace(v - np.mean(v), 50, ax= ax0, title= "Raw_centered signal from extracellular recording electrodes")

    #Filter the signal
    fil_v = su.bandpass_filter(v, 0.5, 110, 1./(sampling_freq * 1000.))
    tpl.plot_nwb_trace(fil_v, 50, ax= ax1, title= "Filtered_signal from extracellular recording electrodes")
    #Apply hilber on the filtered signal
    phase_var, amp_var, freq_var =bt.hilbert_transform_expr(fil_v, ex_delay, ex_dur, 0.02)
    tpl.plot_nwb_trace(phase_var, 50, ax= ax2, title= "Hilbert Phase")

ax0.set_xlim(5,6)
ax1.set_xlim(5,6)
ax2.set_xlim(5,6)
plt.show()
    

####  The plot above shows that the hilbert phase is slightly different for different extracellular recording electrodes
This means that the spike phase is also going to be different depending which electrode we choose to perform the spike phase analysis. Below, I am just plotting all the spike phase values computed using hilbert transorm of each of extracellular recording electrodes.

In [ ]:
sp0 = table[table['sweep_number']==65]['spike_phase'][0]
sp1 = table[table['sweep_number']==65]['spike_phase'][1]
# sp2 = table[table['sweep_number']==163]['spike_phase'][2]
# sp3 = table[table['sweep_number']==163]['spike_phase'][3]
plt.figure(figsize=(20,5))
plt.scatter(np.arange(0,len(sp0)),sp0)
plt.scatter(np.arange(0,len(sp1)),sp1)
# plt.scatter(np.arange(0,len(sp2)),sp2)
# plt.scatter(np.arange(0,len(sp3)),sp3)
plt.ylabel('Phase(Rad)', size=20)
plt.xlabel('Spike', size=20)
plt.show()

#### But anyway, we would like to check if the values computed for spike phase are correct. If everything is fine, then all the scatter points in the plots below MUST cross the orange lines.

In [ ]:
sweep = 65
ex_electrode_id = table['ex_el_id'].unique().tolist()
in_electrode_id = 5
# freq = 31
ex_dur = 13000
ex_delay = 1000
lowcut_freq = 0.5
highcut_freq = 110
dt = 0.02 #ms


for el in ex_electrode_id:
    vi = xu.read_trace_from_nwb(experiment_id, sampling_freq,  sweep, in_electrode_id, saved_data)
    ax = plt.figure(figsize=(20,5))
    ax = plt.subplot(111)
    tpl.plot_nwb_trace(vi, 50, ax= ax)   

    vext = xu.read_trace_from_nwb(experiment_id, sampling_freq,  sweep, el, saved_data)
    filtered_vext = su.bandpass_filter(vext, lowcut_freq, highcut_freq, dt * 0.001)
    phase_var, b, c = bt.hilbert_transform_expr(filtered_vext, ex_delay, ex_dur, dt)
    N = len(phase_var)
    time_step = 1. / (sampling_freq * 1000)
    tstop = (ex_delay + ex_dur)/1000.
    time = np.arange(ex_delay/1000.,tstop , time_step)

    ax.plot(time, [pv - 20. for pv in phase_var])
    spike_tt = table[(table['sweep_number'] == sweep) & (table['ex_el_id']==el)]['spike_tt'][0]
    spike_phase = table[(table['sweep_number'] == sweep) & (table['ex_el_id']==el)]['spike_phase'][0]
    ax.scatter(spike_tt / 1000. , [sp -20 for sp in spike_phase], s=100)
    
    ax.set_xlim(3, 4)
    plt.show()

We corrected the phase and brought all the values between 0 and 2pi. In the figure below, all the points must cross the orange line and also must be at the spike time. We also got rid of all the spikes below 3.5s.

In [ ]:
sweep = 65
ex_electrode_id = table['ex_el_id'].unique().tolist()
in_electrode_id = 5
# freq = 31
ex_dur = 13000
ex_delay = 1000
lowcut_freq = 0.5
highcut_freq = 200
dt = 0.02 #ms


for el in ex_electrode_id:
    print el
    vi = xu.read_trace_from_nwb(experiment_id, sampling_freq,  sweep, in_electrode_id, saved_data)
    ax = plt.figure(figsize=(20,5))
    ax = plt.subplot(111)
    tpl.plot_nwb_trace(vi, 50, ax= ax)   

    vext = xu.read_trace_from_nwb(experiment_id, sampling_freq,  sweep, el, saved_data)
    filtered_vext = su.bandpass_filter(vext, lowcut_freq, highcut_freq, dt * 0.001)
    phase_var, b, c = bt.hilbert_transform_expr(filtered_vext, ex_delay, ex_dur, dt)
    temp = [x + (1.5 * np.pi) for x in phase_var]
    corrected_phase = [(x/(2*np.pi) - int(x/(2*np.pi))) * 2 * np.pi for x in temp]
    
    N = len(phase_var)
    time_step = 1. / (sampling_freq * 1000)
    tstop = (ex_delay + ex_dur)/1000.
    time = np.arange(ex_delay/1000.,tstop , time_step)

    ax.plot(time, [pv - 20. for pv in corrected_phase])
    ax.plot(time, [pv - 20. for pv in phase_var])

    spike_tt = table[(table['sweep_number'] == sweep) & (table['ex_el_id']==el)]['spike_tt_A'][0]
    spike_phase = table[(table['sweep_number'] == sweep) & (table['ex_el_id']==el)]['spike_phase_A_corrected'][0]
    ax.scatter([stt/1000. for stt in spike_tt] , [sp -20 for sp in spike_phase], s=100)
    
    ax.set_xlim(4, 5)
    plt.show()

# Spike phase analysis for one of the conrol experiments

In [ ]:
table[table['ex_amp(nA)']==0]

In [ ]:
table['ex_el_id'].unique().tolist()

In [ ]:
control_sweep = 56
other_sweep = [ 57, 58, 59, 60 ]
freq = [1, 5, 9, 31]
ex_electrode_id = table['ex_el_id'].unique().tolist()[1]
# ex_electrode_id= 4
in_electrode_id = 5
ex_dur = 13000
ex_delay = 1000
lowcut_freq = 0.5
highcut_freq = 200
dt = 0.02 #ms


for s in other_sweep:
        print "freq",freq[other_sweep.index(s)]
        vi = xu.read_trace_from_nwb(experiment_id, sampling_freq,  control_sweep, in_electrode_id, saved_data)
        ax = plt.figure(figsize=(20,5))
        ax = plt.subplot(111)
        tpl.plot_nwb_trace(vi, 50, ax= ax)   
        vext = xu.read_trace_from_nwb(experiment_id, sampling_freq,  s, ex_electrode_id, saved_data)
        filtered_vext = su.bandpass_filter(vext, lowcut_freq, highcut_freq, dt * 0.001)
        phase_var, b, c = bt.hilbert_transform_expr(filtered_vext, ex_delay, ex_dur, dt)
        temp = [x + (1.5 * np.pi) for x in phase_var]
        corrected_phase = [(x/(2*np.pi) - int(x/(2*np.pi))) * 2 * np.pi for x in temp]
    
        N = len(phase_var)
        time_step = 1. / (sampling_freq * 1000)
        tstop = (ex_delay + ex_dur)/1000.
        time = np.arange(ex_delay/1000.,tstop , time_step)

        ax.plot(time, [pv - 20. for pv in corrected_phase])
        ax.plot(time, [pv - 20. for pv in phase_var])

        spike_tt = table[(table['sweep_number'] == control_sweep) & (table['ex_el_id']==ex_electrode_id) & (table['ex_frequency']==freq[other_sweep.index(s)])]['spike_tt_A'][0]
        spike_phase = table[(table['sweep_number'] == control_sweep) & (table['ex_el_id']==ex_electrode_id)  & (table['ex_frequency']==freq[other_sweep.index(s)])]['spike_phase_A_corrected'][0]
        ax.scatter([stt/1000. for stt in spike_tt] , [sp -20 for sp in spike_phase], s=100)
    
        ax.set_xlim(3, 5)
        plt.show()

# Can you think of any other tests? If yes, let me know

in the table, all the columns which have the "_A" in their name, they are the values which are cut from the total amount for the analysis. For example Spike_tt, is the spike threshold time for all the spikes in one experiment. However spike_tt_A is the spike threshold time for only the spikes between 2 and 12 second for which we are doing the analysis.

In [ ]:
temp =table.apply(lambda row: [x  for x in row['spike_phase_A_corrected'] if (x > 2 * np.pi or x < 0)], axis=1)
if np.sum([len(l) for l in temp]) == 0:
    print "Good"
else:
    print "Some of spike phases are out of range"


# How are we filtering: This is an example

In [ ]:
Fs = 8000
f1 = 20
f2 = 80
sample = 8000
t = np.arange(sample)
dt = 1. / Fs 
signal = np.sin(2 * np.pi * f1 * t / Fs) + np.sin(2 * np.pi * f2 * t / Fs)

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(signal)
plt.show()

plt.figure(figsize=(20,5))
sig_fft, sampling_freq = su.compute_fft(signal, dt)
tpl.plot_fft(sig_fft, sampling_freq)
plt.show()

filtered_sig =  su.bandpass_filter(signal,-50, 50, dt)
plt.figure(figsize=(20,5))
plt.plot(filtered_sig- signal)
plt.show()
plt.figure(figsize=(20,5))
plt.plot(filtered_sig)
plt.plot(signal)
plt.show()

sig_fft, sampling_freq = su.compute_fft(filtered_sig, dt)
tpl.plot_fft(sig_fft, sampling_freq)
plt.show()

# This is filtered signal if we cut anything between 0.5 and 20000

In [ ]:
fil = bandpass_filter(signal = Ve,lowcut_freq=0.5, highcut_freq=20000, time_step=1./50000)
plt.figure(figsize=(20,5))
plt.plot(Ve-fil)
plt.show()

plt.figure(figsize=(20,5))
plt.plot(Ve)
plt.plot(fil)
plt.show()

a, b = compute_fft(Ve, 1./50000)
plot_fft(a, b)
plt.show()

a, b = compute_fft(fil, 1./50000)
plot_fft(a, b)
plt.show()